In [ ]:
from src.tasks.infer import infer_from_trained

In [ ]:
class D:
    pass
        
args = D()
args.model_no = 0
args.model_size = 'bert-base-uncased'
args.num_classes = 4 # TODO: Important: Change for nyt_mix_news to 4. NYT - 3

task = 'classification' # classification for proba / infer_one_sentence_labels

In [ ]:
inferer = infer_from_trained(args, detect_entities=False, load_best=True, task=task)

## Extract Embeddings from BERT

In [ ]:
import pandas as pd

def get_sent_embs(sent):
    inferer.net.task = 'fewrel'
    return inferer.get_one_sent_output(sent)[0].cpu().numpy()    

def get_all_embs(df,data_path,inferer):
    ddf = df
    if ddf is None:
        ddf = pd.read_pickle(data_path)
    
    ddf = ddf[ddf.sents.str.len() < inferer.tokenizer.max_len]
            
    ddf['emb'] = ddf.sents.apply(get_sent_embs)    
    return ddf

def convert_emb_to_df_cols(df):    
    new_df = pd.concat([df.drop(columns='emb'),pd.DataFrame(df.emb.tolist())],axis=1)
    cols = list(set(new_df.columns) - set(['sents', 'relations', 'relations_id']))
    return new_df, cols

In [ ]:
df_test = get_all_embs(None, './data/df_test.pkl',inferer)
df_test,cols = convert_emb_to_df_cols(df_test.reset_index(drop=True))
df_train = get_all_embs(None, './data/df_train.pkl',inferer)
df_train,_ = convert_emb_to_df_cols(df_train.reset_index(drop=True))

## Find similar samples

In [ ]:
import sys
sys.path += ['/home/dekelco/views/dsutil/']

In [ ]:
from error_analysis.similar_samples import SimilarSamples

In [ ]:
ss = SimilarSamples(df_train, columns=cols)

In [ ]:
df_sim = ss.merge_similar_rows(df_test[10:20], k=2).reset_index()

In [ ]:
s = df_sim.style
tt = pd.DataFrame(data=df_sim.sents_x.tolist(), columns=['sents_x'], index=df_sim.index)
s.set_tooltips(tt)
df_sim2 = df_sim.filter(regex=("index|sents|relations|relations_id"))
df_sim2.head(20)

In [ ]:
n=11
print(df_sim2.iloc[n].sents_x)
print()
print(df_sim2.iloc[n].sents_y)

## Predict single sample

In [ ]:
sent = "The CEO of [E1] IBM [\E1] , [E2] Steve Cohen [\E2] , indicated that he is contributing 12 million Euros in aid this year "
inferer.net.task = 'classification'
preds = inferer.infer_one_sentence_labels(sent)

emb = get_sent_embs(sent)
df_single = pd.DataFrame(data=[sent],columns=['sents'])
df_single = pd.concat([df_single,pd.DataFrame(data=[emb.tolist()])],axis=1)


df_sim3 = ss.merge_similar_rows(df_single, k=1).reset_index()
print('**sentence**')
print(df_sim3.iloc[0].sents_x)
print('\n**most similar**')
print(df_sim2.iloc[0].sents_y)
print(f'\n**similar class={df_sim2.iloc[0].relations_y}**')
preds

### Spacy

In [ ]:
import spacy
nlp = spacy.load('en_core_web_trf')

In [ ]:
sent = "European Investment Bank President ,Warner Heuer"
sents_doc = nlp(sent)
ents = sents_doc.ents # get entities
for ent in ents:
    print(ent.label_, ent.text)